Download template MNI-HCP which Tian atlas is defined in from templateflow

In [ ]:
from templateflow import api
api.get('MNI152NLin6Asym', resolution=1, suffix='T1w')

Plot tian atlas in different scales

In [ ]:
%matplotlib qt
import matplotlib.pyplot as plt
from subcortex_visualization.plotting import plot_subcortical_data

for s in ["S1", "S2"]:
    plot_subcortical_data(atlas=f"Melbourne_{s}", hemisphere='both', cmap="hsv")
    fig = plt.gcf()
    ax = plt.gca()
    fig.patch.set_facecolor('black')
    ax.set_facecolor('black')         
    plt.show()

Visualizing connectome matrix

In [1]:
%matplotlib qt
import matplotlib.pyplot as plt
from cmap import Colormap
from matplotlib.colors import LinearSegmentedColormap
from nilearn.plotting import plot_matrix
import pandas as pd
import numpy as np

In [27]:
df = pd.read_csv("/Users/payamsadeghishabestari/temp_folder/dti/asjt/conn/tian_S3_conn.csv", header=None)
cm = Colormap('colorbrewer:OrRd_9')
combined_cmap = LinearSegmentedColormap.from_list("", cm(np.linspace(0, 1, 500)), N=1000)

labels_fname = "./data/Tian2020MSA/3T/Subcortex-Only/Tian_Subcortex_S3_3T_label.txt"
labels = np.loadtxt(labels_fname, dtype=str)

fig, ax = plt.subplots(figsize=(10, 8))
plot_matrix(df, labels=labels, cmap=combined_cmap, axes=ax, colorbar=True, vmax=0.9, vmin=0)

for label in ax.get_xticklabels()[:25]:
    label.set_color('lightgreen')
for label in ax.get_yticklabels()[:25]:
    label.set_color('lightgreen')

for label in ax.get_xticklabels()[25:]:
    label.set_color('lavender')
for label in ax.get_yticklabels()[25:]:
    label.set_color('lavender')

fig.patch.set_facecolor('black')
ax.set_facecolor('black')          

# Make grid lines white for visibility (optional)
for spine in ax.spines.values():
    spine.set_color('white')


plt.show()

Convert dataset into BIDS (anat and func)

In [5]:
import os
import shutil

# Paths
fMRI_path = "/Volumes/Extreme_SSD/payam_data/antinomics_data/fMRI"
sMRI_path = "/Volumes/Extreme_SSD/payam_data/antinomics_data/sMRI_T1"
bids_path = "/Volumes/Extreme_SSD/payam_data/antinomics_data/BIDS"

# Create BIDS dataset folder if it doesn't exist
os.makedirs(bids_path, exist_ok=True)

# Create minimal dataset_description.json
import json
dataset_description = {
    "Name": "My fMRI/sMRI Dataset",
    "BIDSVersion": "1.8.0",
    "DatasetType": "raw"
}
with open(os.path.join(bids_path, "dataset_description.json"), "w") as f:
    json.dump(dataset_description, f, indent=4)

# Helper to zero-pad subject IDs
def sub_id(idx):
    return f"sub-{idx:02d}"

# List of subjects based on sMRI filenames
subjects = [f.replace(".nii.gz", "") for f in sorted(os.listdir(sMRI_path)) if f.endswith(".nii.gz")]

for i, subj_name in enumerate(subjects, start=1):
    sub_folder = os.path.join(bids_path, sub_id(i))
    
    # Create anat folder
    anat_folder = os.path.join(sub_folder, "anat")
    os.makedirs(anat_folder, exist_ok=True)
    
    # Copy T1 (anat)
    t1_src = os.path.join(sMRI_path, f"{subj_name}.nii.gz")
    t1_dst = os.path.join(anat_folder, f"{sub_id(i)}_T1w.nii.gz")
    shutil.copy(t1_src, t1_dst)
    
    # Create func folder
    func_folder = os.path.join(sub_folder, "func")
    os.makedirs(func_folder, exist_ok=True)
    
    # Copy fMRI scans from all session folders into runs
    run_counter = 1
    for session_folder in ["s1", "s2"]:
        func_src_folder = os.path.join(fMRI_path, session_folder)
        func_files = [f for f in os.listdir(func_src_folder) if f.startswith(subj_name) and f.endswith(".nii.gz")]
        for func_file in sorted(func_files):
            func_src = os.path.join(func_src_folder, func_file)
            func_dst = os.path.join(func_folder, f"{sub_id(i)}_task-rest_run-{run_counter:02d}_bold.nii.gz")
            shutil.copy(func_src, func_dst)
            run_counter += 1

CheckPTA

In [2]:
from pathlib import Path
import pandas as pd
import numpy as np
from scipy.io import loadmat

# --- Settings ---
df = pd.read_excel("./data/master.xlsx")
base_dir = Path("/Users/payamsadeghishabestari/temp_folder/audiometry")
df["PTA_L"] = np.nan
df["PTA_R"] = np.nan
pta_freqs = [500, 1000, 2000]

# for subject in df["subject_ID"].values:
subject = "erzw"
subj_dir = base_dir / subject   

for hemi in ["L", "R"]:
    files = [
                f for f in subj_dir.glob("*.mat")
                if f.name.lower().startswith(f"{subject.lower()} {hemi.lower()}")
            ]
    if not files:
        raise ValueError(f"{subject} has missing audiometry files.")

    fname = files[0]
    data = loadmat(fname)
#         freqs = data["betweenRuns"]["var1Sequence"][0][0][0]
#         thrs  = data["betweenRuns"]["thresholds"][0][0][0]
#         order = np.argsort(freqs)
#         freqs_sorted = freqs[order]
#         thrs_sorted  = thrs[order]
#         mask = np.isin(freqs_sorted, pta_freqs)
#         if subject == "typy":
#             mask = np.isin(freqs_sorted, [1000, 2000])

#         if not mask.any():
#             raise ValueError(f"{subject} has missing frequencies.")

#         pta = thrs_sorted[mask].mean()
#         df.loc[df["subject_ID"] == subject, f"PTA_{hemi}"] = pta

# df["PTA"] = 0.5 * (df["PTA_L"] + df["PTA_R"])
# df = df[['subject_ID', 'group', 'age', 'sex', 'PTA']]

In [4]:
data["betweenRuns"]["var1Sequence"][0][0][0]

array([ 1000,   250,   500,  2000,  6000, 12000,  4000,   125,  8000],
      dtype=uint16)

In [5]:
data["betweenRuns"]["thresholds"][0][0][0]

array([ -7.25,  -1.75, -11.25, -12.75,   6.75,  18.  ,  -5.75, -10.  ,
         3.75])

In [40]:
import os
from pathlib import Path
from tqdm import tqdm

In [ ]:
data_dir = Path("/Volumes/G_USZ_ORL$/Research/ANTINOMICS/data")
eeg_dir = data_dir / "eeg"
audio_dir = data_dir / "audiometry"
files = [f for f in eeg_dir.iterdir() if f.is_file() and f.name.endswith("_rest.vhdr")]
files = sorted(files, key=os.path.getctime)
subject_tide_ids = [str(i) for i in range(70001, 70001 + len(files))]
subject_antinomics_ids = [file.stem[:4] for file in files]
subject_to_tide_map = dict(zip(subject_antinomics_ids, subject_tide_ids))

In [46]:
for subject in tqdm(subject_antinomics_ids):
    subject_audio_dir = audio_dir / subject
    if subject_audio_dir.exists():
        for fname in subject_audio_dir.iterdir():
            for hemi in ["L", "R"]:
                if fname.name.lower().startswith(f"{subject.lower()} {hemi.lower()} ant"):
                    data = loadmat(fname)
                    freqs = data["betweenRuns"]["var1Sequence"][0][0][0]
                    thrs  = data["betweenRuns"]["thresholds"][0][0][0]
                    order = np.argsort(freqs)
                    freqs_sorted = freqs[order]
                    thrs_sorted  = thrs[order]
                    

100%|██████████| 72/72 [00:11<00:00,  6.15it/s]


In [ ]:
import numpy as np
import pandas as pd
from scipy.io import loadmat
from tqdm import tqdm

# The frequencies you want as columns
target_freqs = [125, 250, 500, 1000, 2000, 4000, 6000, 8000, 12000]

# Build all hemi-specific column names
freq_cols = []
for hemi in ["L", "R"]:
    for f in target_freqs:
        freq_cols.append(f"{hemi}_{f} Hz")

rows = []

for subject in tqdm(subject_antinomics_ids):
    
    subject_audio_dir = audio_dir / subject
    tide_id = subject_to_tide_map.get(subject, np.nan)

    # Create empty row for subject
    row = {"antinomics_id": subject, "tide_id": tide_id}

    # Initialize all possible columns with NaN so missing ones remain NaN
    for col in freq_cols:
        row[col] = np.nan

    if subject_audio_dir.exists():
        for fname in subject_audio_dir.iterdir():
            for hemi in ["L", "R"]:
                if fname.name.lower().startswith(f"{subject.lower()} {hemi.lower()}"):

                    data = loadmat(fname)
                    freqs = data["betweenRuns"]["var1Sequence"][0][0][0]
                    thrs  = data["betweenRuns"]["thresholds"][0][0][0]

                    # Sort
                    order = np.argsort(freqs)
                    freqs_sorted = freqs[order]
                    thrs_sorted  = thrs[order]

                    # Fill the corresponding hemi columns
                    for f, thr in zip(freqs_sorted, thrs_sorted):
                        f_int = int(f)
                        col_name = f"A{hemi}E_{f_int}"
                        if col_name in row:
                            row[col_name] = thr

    rows.append(row)

# Create DataFrame
df = pd.DataFrame(rows)

# Order columns
df = df[["antinomics_id", "tide_id"] + freq_cols]

100%|██████████| 72/72 [00:06<00:00, 11.88it/s]
